In [ ]:

!pip install transformers
!pip install pandas scikit-learn torch


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import torch
from transformers import BertTokenizer, BertModel
import warnings
warnings.filterwarnings('ignore')


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/MINI PROJECT/DATASET/augmented_dataset1.csv')
print(df.tail())

df = pd.concat([
    df[df['Label'] == 1].sample(n=300, random_state=42),
    df[df['Label'] == 0].sample(n=300, random_state=42),
    df[df['Label'] == 2].sample(n=300, random_state=42)
])

df = df.reset_index(drop=True)
print(df.head())


         PID                                          Text data  Label
18052  18052  ov trudging remembe dehydrated pregame adays d...      2
18053  18053  exacerbate floated prescribes wonky combo fibr...      2
18054  18054  voice heavy terrible awake help need night its...      2
18055  18055  wonder doesn smile face hate what right can be...      2
18056  18056  redacted overshare detail center colleges room...      2
    PID                                          Text data  Label
0  2868  my to have about being in not any way and im i...      1
1  5924  for to the know but that in few of broke up wa...      1
2  3764  new being same dont this shit will so me think...      1
3  4144  for my to have once the new year there anymore...      1
4  2780  for my to have the new year anymore about but ...      1


In [ ]:
model_class, tokenizer_class, pretrained_weights = (BertModel, BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


In [ ]:
def get_bert_features(text_batch, model, tokenizer):
    encoded_inputs = tokenizer.batch_encode_plus(
        text_batch,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded_inputs['input_ids']
    attention_mask = encoded_inputs['attention_mask']

    with torch.no_grad():
        last_hidden_states = model(input_ids, attention_mask=attention_mask)

    features = last_hidden_states[0][:, 0, :].numpy()
    return features


In [ ]:
batch_size = 100
features_list = []
for i in range(0, df.shape[0], batch_size):
    text_batch = df['Text data'].iloc[i:i+batch_size].tolist()
    batch_features = get_bert_features(text_batch, model, tokenizer)
    features_list.append(batch_features)

features = np.concatenate(features_list, axis=0)
labels = df['Label'].values


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [ ]:
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(x_train, y_train)

pred = dt_clf.predict(x_test)
print(classification_report(y_test, pred))


              precision    recall  f1-score   support

           0       0.68      0.57      0.62        67
           1       0.65      0.65      0.65        60
           2       0.62      0.75      0.68        53

    accuracy                           0.65       180
   macro avg       0.65      0.66      0.65       180
weighted avg       0.65      0.65      0.65       180

